Sistemas administradores de bases de datos: hay muchos (mysql, sqlserver, postreSQL). son en general programas pesados, con muchas funcionalidades, pensados en aplicaciones grandes.
Para nuestro caso, no necesitamos tanto, por lo que utilizaremos SQLite, un administrador altamente compacto y funcional, compatible con SQL. SQLite es compatible con Windows, Linux y Mac, mientras que su instalación es trivial para cualquiera de estas tres plataformas.



In [1]:
import sqlite3

conn = sqlite3.connect('example.db')
c = conn.cursor()

# Create table
c.execute("CREATE TABLE stocks(date text, trans text, symbol text, qty real, price real)")

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
c.execute("INSERT INTO stocks VALUES ('2006-01-06','SELL','THOSE',50,10.5)")

# Save (commit) the changes
conn.commit()

Podemos verificar la creación de la tabla utilizando el siguiente comando:

In [2]:
c.execute('PRAGMA table_info([stocks])')
print(c.fetchall())

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

[(0, 'date', 'text', 0, None, 0), (1, 'trans', 'text', 0, None, 0), (2, 'symbol', 'text', 0, None, 0), (3, 'qty', 'real', 0, None, 0), (4, 'price', 'real', 0, None, 0)]


Como se puede ver, el comando anterior nos entrega información relvante, como tipo de dato y posición, para cada una de las columnas de la tabla.

Por otro lado, si queremos verificar que la inserción fue realizada correctamente, debemoos una consulta sobre la misma base de datos. Es importante notar que en este momento, no existe una conexión activa con la base de datos *ejemplo*, ya que esta fue cerrada con el comando **connection.close()**. De esta manera, antes de realizar una consulta, es necesario abrir nuevamente una conexión, la que a diferencia del caso anterior, no deberá crear un nuevo archivo *ejemplo.db*, ya que este fue creado anteriormente. Es importante notar que no es necesario cerrar siempre la conexión con la base de datos, sólo cuando no se espera interactuar con ella en el plazo inmediato.

In [3]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute('SELECT * FROM stocks')
#obtenemos sólo una fila, si queremos todas, debemos usar fetchall
print(c.fetchone())
conn.close()

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


Otra manera de revisar los contenidos de la base de datos, es utilizando una herramienta especializada para esto. Dentro de la gran variedad existente, una opción simple y liviana es SQLiteStudio, que también es multiplataforma (Windows, Linux, Mac). 

Como se aprecia en la siguiente imagen, utilizando SQLiteStudio, es posible explorar visualmente el contenido de las bases de datos de SQLite.

Un error muy común en la gran mayoría de los programas que interactuan con bases de datos, es la utilización directa de *strings* para introducir información en las consultas. Esto puede generar grandes problemas de seguridad, debido a un ataque conocido como SQL Injection, que consiste en introducir, como dato para una consulta, consultas completas de SQL que pueden causar grandes pérdidas en la base de datos. Veamos el siguiente caso como ejemplo, donde eliminaremos la tabla *stocks* utilizando esta técnica:

In [4]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
symbol = "'RHAT'; DROP TABLE stocks;"
c.executescript("SELECT * FROM stocks WHERE symbol = %s" % symbol)

#verificamos la existencia de la tabla
c.execute('PRAGMA table_info([stocks])')
print(c.fetchall())

[]


Para evitar esto, es fundamental utilizar parameter substitution. Put ? as a placeholder wherever you want to use a value, and then provide a tuple of values as the second argument to the cursor’s execute() method

In [5]:
c.execute("CREATE TABLE stocks(date text, trans text, symbol text, qty real, price real)")
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?', t)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


Muchas veces, deberemos insertar más de un valor simultaneamente en la base de datos. Con el fin de evitar el llamado repetido a la función execute, SLite provee la función executemany que permite ejecutar múltiples consultas de manera simultánea:

In [ ]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

Una manera más limpia de verificar los resultados obtenidos por una consulta select, es utilizar el cursor resultante de una consulta como un iterador:

In [ ]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)